# Week 3: Modeling the Ocean

**Discussion**: Revisit the emissions pathways we introduced last week. On the radiative forcing plot, draw a pathway you think our world is likely to follow. Compare with your classmates and discuss your choice.

This week, we'll discuss how including the ocean impacts energy balance models, using a two-layer model introduced in [this paper](https://eartharxiv.org/repository/view/4581/). **Quantitative** students will model the ocean and atmosphere as a system of differential equations. **Qualitative** students will assess their confidence in the outputs of the models we developed in this lab.

## Two-Layer Ocean Model (Quant.)

In [ ]:
import numpy as np # Mathematical functions
import matplotlib.pyplot as plt # Making plots
import pandas as pd # Tabular data

def euler(time, initials, derivatives, *extra_inputs): # provided for you
    from inspect import signature

    # Validate time input
    try: time.iloc[0]
    except AttributeError: time = pd.Series(time)
    if len(time) < 2:
        raise ValueError("must have at least two time points to use Euler's method")
    for t in time:
        try: float(t)
        except ValueError: raise TypeError('time points must be numbers (' + str(t) + ' is not a number)')

    # Validate initials input
    try: initials.iloc[0]
    except AttributeError: initials = pd.Series(initials)
    for initial in initials:
        try: float(initial)
        except ValueError: raise TypeError('initial conditions must be numbers (' + str(initial) + ' is not a number)')

    # Validate derivatives input
    try: derivatives.iloc[0]
    except AttributeError: derivatives = pd.Series(derivatives)
    if len(initials) != len(derivatives):
        raise ValueError('number of initial conditions is ' + str(len(initials)) + ', but number of differential equations is ' + str(len(derivatives)))
    for deriv in derivatives:
        if not callable(deriv):
            raise TypeError('differential equations must be functions (' + str(deriv) + ' is not a function)')
        if len(deriv.__name__) < 5 or deriv.__name__[0] != 'd' or deriv.__name__[-3:] != '_dt':
            raise ValueError('differential equations must be named like dX_dt, not ' + deriv.__name__)
    for (name, params) in [(deriv.__name__, list(signature(deriv).parameters.keys())) for deriv in derivatives]:
        if len(params) < 1 + len(derivatives):
            raise TypeError('each differential equation must have 1 input for time plus ' + str(len(derivatives)) + ' input(s) for the current values of each variable')
        if params[0] != 't':
            raise TypeError('the first input of ' + name + ' must be time (t)')
        for i in range(len(derivatives)):
            if params[i + 1] != derivatives.iloc[i].__name__[1:-3]:
                raise TypeError('input #' + str(i + 1) + ' of ' + name + ' must be ' + derivatives.iloc[i].__name__[1:-3])
        if len(params) != 1 + len(derivatives) + len(extra_inputs):
            raise TypeError('euler was given ' + str(len(extra_inputs)) + ' extra input(s), but ' + name + ' has ' + str(len(params) - len(derivatives) - 1) + ' extra input(s)')

    # Initialize output array and set initial conditions
    names = [deriv.__name__[1:-3] for deriv in derivatives]
    variables = {name: pd.Series(np.zeros(len(time)), index=time) for name in names}
    for i in range(len(names)):
        variables[names[i]].loc[time[0]] = initials.iloc[i]

    # Step through output array, perform Euler's method
    for t in range(0, len(time) - 1):
        delta_t = time.iloc[t+1] - time.iloc[t]
        current = [variables[name].loc[time.iloc[t]] for name in names]
        for i in range(len(names)):
            deriv = derivatives.iloc[i](time.iloc[t], *current, *extra_inputs)
            try: float(deriv)
            except ValueError: raise TypeError('differential equations must output numbers (' + str(deriv) + ' is not a number)')
            variables[names[i]].loc[time.iloc[t+1]] = variables[names[i]].loc[time.iloc[t]] + delta_t * deriv
    return pd.DataFrame(variables)

Read pages 8-9 of the paper linked above. The top layer represents the Earth's surface and the upper ocean combined; the bottom layer represents the deep ocean. Make an autonomous model in Python that captures the important parts of the paper's model—it will need to be a system of differential equations. Then, make plots of the model's output for a few different initial conditions. To use the Euler method with a system of differential equations, you just have to put the initial conditions and differential equations in brackets, separated by commas. For example:

    euler(t, [T1_initial, T2_initial], [dT1_dt, dT2_dt])

In [ ]:
# TO-DO: Define necessary constants


In [ ]:
# TO-DO: Define the differential equations


In [ ]:
# TO-DO: Plot output


Consider all the parameters to the model you defined as constants. All of these except for the Stefan-Boltzmann constant could vary (at least in theory), and our measurements of them are subject to error. Modify each parameter to see how its variation affects the model output. Write down each parameter's effect, including how the magnitude of the effect differs between parameters.

Find a reasonable set of parameter values that gives results similar to the real Earth. Explain how you arrived at your parameter choices.

## Ocean Heat Uptake (Qual.)

<figure>
    <img src="Lab1Images/fig7.png" width="400"/>
    <figcaption>
        <a href="https://eartharxiv.org/repository/view/4581/">
            Fig. 7: Energy balance diagram including the ocean (fig. 2 in the paper)
        </a>
    </figcaption>
</figure>

In around 200 words, answer the following questions about Figure 7.
- Explain the different components of the model. Intuitively, what does each term represent?
- Which heats up faster, land or ocean? Why?
- What do we gain from including the ocean? Would it be worthwhile to include more ocean layers? What about atmospheric layers?

## Essay: Simplified Models (Qual.)

In around 1000 words, explain how much we can trust climate models' predictive power. Consider the first two chapters of *Computing the Climate*, as well as the model we developed in this lab. You might want to ask a quantitative student to show you their work in order to gather more information.